In [ ]:
push!(LOAD_PATH, pwd())

using PyPlot, Statistics, CPUTime
using SpinBodies, MBTrees;

In [ ]:
const maxsteps = 1000000
const N        = 50
const T_step   = 0.1
const Ts       = T_step:T_step:4;

In [ ]:
freeman_data      = Vector{Float64}(undef, length(Ts))
freeman_fast_data = Vector{Float64}(undef, length(Ts))
metropolis_data   = Vector{Float64}(undef, length(Ts));

# Freeman Data

In [ ]:
for (i, T) in enumerate(Ts)
    lattice = SpinLattice(N, T)
    tree = build_tree(lattice)
    times = Vector{Float64}(undef, 20)
    for j = 1:20
        time = @CPUelapsed while lattice.steps < maxsteps
            freeman_step!(lattice, tree)
        end
        times[j] = time
    end
    freeman_data[i] = mean(times)
    if i % 5 == 0
        println("T = $T")
    end 
end

# Freeman (Fast) Data

In [ ]:
for (i, T) in enumerate(Ts)
    lattice = SpinLattice(N, T)
    tree = build_tree(lattice)
    times = Vector{Float64}(undef, 20)
    for j = 1:20
        time = @CPUelapsed while lattice.steps < maxsteps
            freeman_step!(lattice, tree, fast=true)
        end
        times[j] = time
    end
    freeman_fast_data[i] = mean(times)
    if i % 5 == 0
        println("T = $T")
    end 
end

# Metropolis Data

In [ ]:
for (i, T) in enumerate(Ts)
    lattice = SpinLattice(N, T)
    times = Vector{Float64}(undef, 20)
    for j = 1:20
        time = @CPUelapsed while lattice.steps < maxsteps
            metropolis_step!(lattice)
        end
        times[j] = time
    end
    metropolis_data[i] = mean(times)
    if i % 5 == 0
        println("T = $T")
    end 
end

In [ ]:
semilogy(Ts, freeman_data ./ metropolis_data) # / freeman_data[end] * metropolis_data[end])
semilogy(Ts, freeman_fast_data ./ metropolis_data) # / freeman_fast_data[end] * metropolis_data[end])
legend(["freeman_CPUtime / metropolis_CPUtime", "freeman_fast_CPUtime / metropolis_CPUtime"])
# savefig("freeman_CPUtime_per_metropolis_CPUtime.pdf")